In [1]:
%cd ~/RATER-D

/home/daved/RATER-D


In [2]:
import pandas as pd
import janitor
import numpy as np
from Levenshtein import distance as levenshtein_distance

In [3]:
train = pd.read_excel('data/processed/train_val_test.xlsx', sheet_name = 'train').clean_names()
val   = pd.read_excel('data/processed/train_val_test.xlsx', sheet_name = 'val').clean_names()
test  = pd.read_excel('data/processed/train_val_test.xlsx', sheet_name = 'test').clean_names()

In [4]:
dat = pd.concat([train, val, test])
dat_journal_subset = dat.query("journalname == 'Academy of Management Journal' or journalname == 'Journal of Applied Psychology'")

In [5]:
dat_journal_subset.groupby('journalname').size()

journalname
Academy of Management Journal    12290
Journal of Applied Psychology    46552
dtype: int64

In [6]:
dat_journal_subset.year.describe()

count    58842.000000
mean      2004.343479
std          4.029663
min       1976.000000
25%       2001.000000
50%       2005.000000
75%       2007.000000
max       2012.000000
Name: year, dtype: float64

In [7]:
test_journal_subset = test.query("journalname == 'Academy of Management Journal' or journalname == 'Journal of Applied Psychology'")

In [8]:
test_journal_subset.groupby('journalname').size()

journalname
Academy of Management Journal    1364
Journal of Applied Psychology    4368
dtype: int64

In [9]:
test_journal_subset.year.describe()

count    5732.000000
mean     2003.676727
std         4.216112
min      1998.000000
25%      2000.000000
50%      2003.000000
75%      2007.000000
max      2012.000000
Name: year, dtype: float64

In [10]:
test_all_journals = test

In [11]:
test_all_journals.groupby('journalname').size()

journalname
Academy of Management Journal                   1364
American Educational Research Journal           1458
American Journal of Sociology                      6
American Sociological Review                     116
Annals of Behavioral Medicine                    108
Educational Evaluation and Policy Analysis      6960
Health Education Research                        836
Information Systems Research                    2971
Journal of Applied Psychology                   4368
Journal of Consumer Research                     297
Journal of Management Information Systems       5352
Journal of Marketing Research                    560
Journal of Nursing Scholarship                   160
Journal of Personality and Social Psychology    2980
MIS Quarterly                                   4738
Nursing Research                                 220
Research in Nursing and health                   503
Social Forces                                    214
Sociology of Education            

In [12]:
test_all_journals.year.describe()

count    33304.000000
mean      2005.736548
std          4.997097
min       1987.000000
25%       2003.000000
50%       2007.000000
75%       2007.000000
max       2020.000000
Name: year, dtype: float64

In [13]:
train = pd.read_excel("data/raw/Highest domain-fixed.xlsx", sheet_name = 3)
pred_df = pd.read_excel("data/raw/Sample data sheet for Kai v9.xlsx")

human_sheet = pred_df.merge(train, on = 'Match ID', how = 'right')
human_sheet['Full construct'] = human_sheet['Construct name'] + ': ' + human_sheet['Construct definition']

human_sheet = human_sheet.clean_names()
len(human_sheet)

858

In [14]:
llm_dat_defs = human_sheet.construct_definition.str.lower().str.strip().unique()
test_dat_defs = dat.definition.str.lower().str.strip().str.encode('latin1').str.decode('cp1252').unique()

for each_def in llm_dat_defs:
    if each_def in test_dat_defs:
        print('Match')

In [15]:
distance_threshold = 1
search = True

while search:
    for each_def_a in llm_dat_defs:
        for each_def_b in test_dat_defs:
            if levenshtein_distance(each_def_a, each_def_b) <= distance_threshold:
                print('Match at distance threshold of', distance_threshold)
                search = False
                break
        if not search:
            break
    distance_threshold += 1

Match at distance threshold of 12


In [16]:
llm_dat_items = human_sheet.item_text.str.lower().str.strip().unique()
test_dat_items = test.itemtext.str.lower().str.strip().str.encode('latin1').str.decode('cp1252').unique()

for each_item in llm_dat_items:
    if each_item in test_dat_items:
        print('Match')

In [17]:
distance_threshold = 1
search = True

while search:
    for each_item_a in llm_dat_items:
        for each_item_b in test_dat_items:
            if levenshtein_distance(each_item_a, each_item_b) <= distance_threshold:
                print('Match at distance threshold of', distance_threshold)
                search = False
                break
        if not search:
            break
    distance_threshold += 1

Match at distance threshold of 7
